In [1]:
import chromadb, requests
from pprint import pprint
from app.models import ChromaDbResult
from pydantic import BaseModel
from typing import Any

In [3]:
text = "penalties in the GDPR"
resp = requests.post(
        "http://localhost:8001/embeddings",
        json={"text": text},
        timeout=20
    )
data = resp.json()
emb = data.get("embedding", [])
print(type(emb[0]))

<class 'float'>


In [2]:

chroma_client = chromadb.HttpClient(host="localhost", port=8000)
col = chroma_client.get_collection(name="gdpr")
raw = col.get(where={"article": 9})
#raw = col.query(query_embeddings=[emb], n_results=2)
print(type(raw))
pprint(raw)

<class 'dict'>
{'data': None,
 'documents': [' Processing of special categories of personal data 1.   '
               'Processing of personal data revealing racial or ethnic origin, '
               'political opinions, religious or philosophical beliefs, or '
               'trade union membership, and the processing of genetic data, '
               'biometric data for the purpose of uniquely identifying a '
               'natural person, data concerning health or data concerning a '
               "natural person's sex life or sexual orientation shall be "
               'prohibited. 2.   Paragraph 1 shall not apply if one of the '
               'following applies: (a) the data subject has given explicit '
               'consent to the processing of those personal data for one or '
               'more specified purposes, except where Union or Member State '
               'law provide that the prohibition referred to in paragraph 1 '
               'may not be lifted by the dat

In [5]:
results = [ChromaDbResult(
        id=raw["ids"][0][i],
        document=raw["documents"][0][i], 
        metadata=raw["metadatas"][0][i],
        distance=raw["distances"][0][i]
        ) for i in range(len(raw["ids"][0]))]
print(results)

[ChromaDbResult(id='668080e7-fc12-4087-b04f-fab9cfb9b0c1', document=' (152) Where this Regulation does not harmonise administrative penalties or where necessary in other cases, for example in cases of serious infringements of this Regulation, Member States should implement a system which provides for effective, proportionate and dissuasive penalties. The nature of such penalties, criminal or administrative, should be determined by Member State law.', metadata={'part': 'preamble', 'article_title': '', 'section': 0, 'article': 0, 'section_title': '', 'chapter_title': '', 'chapter': 0}, distance=0.8643079), ChromaDbResult(id='e661cc5a-09b7-4764-a023-b1cccfc2c468', document=' (148) In order to strengthen the enforcement of the rules of this Regulation, penalties including administrative fines should be imposed for any infringement of this Regulation, in addition to, or instead of appropriate measures imposed by the supervisory authority pursuant to this Regulation. In a case of a minor inf

In [11]:
class FunctionDefinition(BaseModel):
    name: str
    description: str = ""
    parameters: dict[str, Any]
    
class Tool(BaseModel):
    type: str = "function"
    function: FunctionDefinition

TOOLS = [
    {"type": "function",
     "function": {
        "name": "gdpr_query",
        "description": "Sends a query related to the GDPR to a vector database, returns relevant results.",
        "parameters": {
         "query_text": "query string goes here."
            }
         }
    },
]
tools = [Tool.model_validate(tool) for tool in TOOLS]
print(tools)

[Tool(type='function', function=FunctionDefinition(name='gdpr_query', description='Sends a query related to the GDPR to a vector database, returns relevant results.', parameters={'query_text': 'query string goes here.'}))]


In [3]:
from pydantic import BaseModel

class TestModel(BaseModel):
    foo: str
    bar: int

m = TestModel(foo="hi", bar=1)

print(m.foo, m.bar)

setattr(m, "_attr", "new attribute!")

print(m._attr)

hi 1
new attribute!
